<a href="https://colab.research.google.com/github/BaBa0525/machine-learning/blob/main/HW5/hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>